#Loading Header Files:

In [ ]:
import re
import os, sys
import random
import pickle
import numpy as np
import numpy.random
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import keras


from moviepy.editor import VideoFileClip, vfx
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model, Sequential
from keras.layers import Input, LSTM, Bidirectional, Dropout, Dense, Activation
from keras_layer_normalization import LayerNormalization
from keras import regularizers
from keras.utils import to_categorical
from keras.utils import np_utils

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)1818624/45929032 bytes (4.0%)5718016/45929032 bytes (12.4%)9723904/45929032 bytes (21.2%)13664256/45929032 bytes (29.8%)17498112/45929032 bytes (38.1%)21405696/45929032 bytes (46.6%)25296896/45929032 bytes (55.1%)29032448/45929032 bytes (63.2%)32890880/45929032 bytes (71.6%)36847616/45929032 bytes (80.2%)40828928/45929032 bytes (88.9%)44859392/45929032 bytes (97.7%)

Using TensorFlow backend.


#Mounting Drive:


In [ ]:
#Mounting the first drive:
from google.colab import drive
drive.mount('/content/drive1')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive1


In [ ]:
#Mounting the second drive:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p /content/drive2
!google-drive-ocamlfuse /content/drive2

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144568 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.19-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.19-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.19-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

#Transfer Learning:

In [ ]:
# c_to_n={"Abuse":0,"Arson":1,"Assault":2,"Burglary":3,"Explosion":4,"Fighting":5,"RoadAccidents":6,"Robbery":7,"Shooting":8,"Shoplifting":9,"Stealing":10,"Vandalism":11,"Normal":12}
c_to_n={"Anomaly":0,"Normal":1}

###Loading InceptionV3:

In [ ]:
# Get model with pretrained weights.
base_model = InceptionV3(
weights='imagenet',
include_top=True)
    
    
# We'll extract features at the final pool layer.
model = Model(
    inputs=base_model.input, 
    outputs=base_model.get_layer('avg_pool').output)


96116736/96112376 [==============================] - 1s 0us/step


###Feature Reduction:

In [ ]:
# images = [] 
# labels_t=[]

def video_frame(path,label):
  
  features=[]
  images = []
  labels_t=[]
  cap = cv2.VideoCapture(path)
  count=0;
  framerate = int(cap.get(cv2.CAP_PROP_FPS))
  i=0;
  count_frame=0
  c=0
  while(True):
      # Capture frame-by-frame
      ret, frame = cap.read()

      if not ret: 
        break
      
#      if frame is None:
#       break;


      # Saves image of the current frame in jpg file
      
      elif count%(1*framerate) == 0 :
        frame = preprocess_input(cv2.resize(frame, (299,299)))
        images.append(frame)
        count_frame+=1;
        # cv2_imshow(frame)
      if(count_frame==8):
        f = model.predict(np.array(images))
        features.append(f)
        labels_t.append(c_to_n[label])
        images = []
        count_frame=0
      
      count+=1;

      
  # When everything done, release the capture
  cap.release()
  cv2.destroyAllWindows()
  
#   f = model.predict(np.array(images))
  
  return features, labels_t

#Creating and Saving Training, Validation and Testing Datasets:
###1. Feature Reduction
###2. Flipping Videos
###3. One-hot encoding

In [ ]:
path=['/content/drive1/My Drive/Training_Set','/content/drive1/My Drive/Validation_Set','/content/drive1/My Drive/Testing_Set']

In [ ]:
labels=['Abuse','Arson','Assault','Burglary','Explosion','Fighting','RoadAccidents','Robbery','Shooting','Shoplifting','Stealing','Vandalism','Normal']
ignore=['.ipynb_checkpoints','Normal_Videos_251_x264.mp4']

In [ ]:
labels_train=[]
transfer_values_train=[]

p=path[0]
dir_list=os.listdir(p)
for dl in dir_list:
    print(str(dl))
    for lab in labels:
      if re.search(lab,str(dl))!=None:
        if lab=='Normal':
          label=lab
          break
        else:
          label='Anomaly'
          break
    features, l=video_frame(p+'/'+str(dl),label)
    labels_train.extend(l)
    print(np.array(features).shape)
    transfer_values_train.extend(np.array(features))
    print(np.array(transfer_values_train).shape)

    #flip the video
    if str(dl) not in ignore:
      clip = VideoFileClip(p+'/'+str(dl))
      reversed_clip = clip.fx(vfx.mirror_x)
      reversed_clip.write_videofile('new_video.mp4')
      features, l=video_frame('new_video.mp4',label)
      transfer_values_train.extend(np.array(features))
      labels_train.extend(l)
      print(np.array(transfer_values_train).shape)

In [ ]:
from keras.utils import np_utils
labels_encoded_train = np_utils.to_categorical(labels_train)

In [ ]:
import pickle

pickle_out=open('/content/drive1/My Drive/Training_Set_values1.pickle',"wb")
pickle.dump(transfer_values_train, pickle_out)
pickle_out.close()

pickle_out=open('/content/drive1/My Drive/Training_Set_labels1.pickle',"wb")
pickle.dump(labels_encoded_train, pickle_out)
pickle_out.close()

In [ ]:
labels_val=[]
transfer_values_val=[]

p=path[1]
dir_list=os.listdir(p)
for dl in dir_list:
    print(str(dl))
    for lab in labels:
      if re.search(lab,str(dl))!=None:
        if lab=='Normal':
          label=lab
          break
        else:
          label="Anomaly"
          break
    features, l=video_frame(p+'/'+str(dl),label)
    labels_val.extend(l)
    print(np.array(features).shape)
    transfer_values_val.extend(np.array(features))
    print(np.array(transfer_values_val).shape)

    #flip the video
    if str(dl) not in ignore:
      clip = VideoFileClip(p+'/'+str(dl))
      reversed_clip = clip.fx(vfx.mirror_x)
      reversed_clip.write_videofile('new_video.mp4')
      features, l=video_frame('new_video.mp4',label)
      transfer_values_val.extend(np.array(features))
      labels_val.extend(l)
      print(np.array(transfer_values_val).shape)

Abuse026.mp4
(5, 8, 2048)
(5, 8, 2048)
[MoviePy] >>>> Building video new_video.mp4
[MoviePy] Writing audio in new_videoTEMP_MPY_wvf_snd.mp3


100%|██████████| 939/939 [00:00<00:00, 1438.74it/s]

[MoviePy] Done.
[MoviePy] Writing video new_video.mp4



100%|██████████| 1278/1278 [00:04<00:00, 314.97it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: new_video.mp4 

(10, 8, 2048)
Abuse027.mp4
(11, 8, 2048)
(21, 8, 2048)
[MoviePy] >>>> Building video new_video.mp4
[MoviePy] Writing audio in new_videoTEMP_MPY_wvf_snd.mp3


...

[MoviePy] Done.
[MoviePy] >>>> Video ready: new_video.mp4 

(3579, 8, 2048)
Copy of Normal_Videos173_x264.mp4
(2, 8, 2048)
(3581, 8, 2048)
[MoviePy] >>>> Building video new_video.mp4
[MoviePy] Writing audio in new_videoTEMP_MPY_wvf_snd.mp3


100%|██████████| 443/443 [00:00<00:00, 2091.31it/s]

[MoviePy] Done.
[MoviePy] Writing video new_video.mp4



100%|██████████| 603/603 [00:01<00:00, 354.76it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: new_video.mp4 

(3583, 8, 2048)


In [ ]:
labels_encoded_val = np_utils.to_categorical(labels_val)

In [ ]:
pickle_out=open('/content/drive1/My Drive/Validation_Set_values1.pickle',"wb")
pickle.dump(transfer_values_val, pickle_out)
pickle_out.close()

pickle_out=open('/content/drive1/My Drive/Validation_Set_labels1.pickle',"wb")
pickle.dump(labels_encoded_val, pickle_out)
pickle_out.close()

In [ ]:
labels_test=[]
transfer_values_test=[]

p=path[2]
dir_list=os.listdir(p)
for dl in dir_list:
    print(str(dl))
    for lab in labels:
      if re.search(lab,str(dl))!=None:
        if lab=='Normal':
          label=lab
          break
        else:
          label='Anomaly'
          break
    features, l=video_frame(p+'/'+str(dl),label)
    labels_test.extend(l)
    print(np.array(features).shape)
    transfer_values_test.extend(np.array(features))
    print(np.array(transfer_values_test).shape)

    #flip the video
    if str(dl) not in ignore:
      clip = VideoFileClip(p+'/'+str(dl))
      reversed_clip = clip.fx(vfx.mirror_x)
      reversed_clip.write_videofile('new_video.mp4')
      features, l=video_frame('new_video.mp4',label)
      transfer_values_test.extend(np.array(features))
      labels_test.extend(l)
      print(np.array(transfer_values_test).shape)

Abuse042.mp4
(10, 8, 2048)
(10, 8, 2048)
[MoviePy] >>>> Building video new_video.mp4
[MoviePy] Writing audio in new_videoTEMP_MPY_wvf_snd.mp3


100%|██████████| 1760/1760 [00:01<00:00, 1364.76it/s]

[MoviePy] Done.
[MoviePy] Writing video new_video.mp4



100%|██████████| 2395/2395 [00:07<00:00, 332.22it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: new_video.mp4 



...

(4077, 8, 2048)
Copy of Normal_Videos110_x264_Trim.mp4
(7, 8, 2048)
(4084, 8, 2048)
[MoviePy] >>>> Building video new_video.mp4
[MoviePy] Writing audio in new_videoTEMP_MPY_wvf_snd.mp3


100%|██████████| 1334/1334 [00:00<00:00, 1348.76it/s]

[MoviePy] Done.
[MoviePy] Writing video new_video.mp4



100%|██████████| 1815/1815 [00:06<00:00, 301.13it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: new_video.mp4 

(4091, 8, 2048)


In [ ]:
labels_encoded_test = np_utils.to_categorical(labels_test)

In [ ]:
pickle_out=open('/content/drive1/My Drive/Testing_Set_values1.pickle',"wb")
pickle.dump(transfer_values_test, pickle_out)
pickle_out.close()

pickle_out=open('/content/drive1/My Drive/Testing_Set_labels1.pickle',"wb")
pickle.dump(labels_encoded_test, pickle_out)
pickle_out.close()